In [1]:
"""
Pour chaque SIRET :
- recherche sur annuaire-entreprises.data.gouv.fr
- onglet Dirigeants
- boucle sur chaque bouton « Ses entreprises »
- extrait tous les SIREN 9 chiffres != SIREN d’origine
⇢ Affiche un récap final et écrit siren_associes.csv
"""

import pandas as pd
import re, csv, time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# ----------------------------------------------------------------------
# 1. PARAMÈTRES UTILISATEUR
# ----------------------------------------------------------------------
CHROMEDRIVER_PATH = r"C:\Users\alfonso.awadalla\Desktop\My Files\chromedriver\chromedriver.exe"  
SIRETS_PATH = r"C:\Users\alfonso.awadalla\Desktop\My Files\siret_missing.csv"
CHROME_BINARY = None                              
HEADLESS = False   

SIRETS = pd.read_csv(SIRETS_PATH, dtype=str).iloc[:, 0].tolist()

OUTFILE = "siren_associes.csv"                         

# ----------------------------------------------------------------------
# 2. OUTILS SELENIUM
# ----------------------------------------------------------------------
def build_driver() -> webdriver.Chrome:
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")  # Chrome ≥124
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    if CHROME_BINARY:
        opts.binary_location = CHROME_BINARY

    service = Service(executable_path=CHROMEDRIVER_PATH)
    driver  = webdriver.Chrome(service=service, options=opts)
    driver.set_window_size(1280, 900)
    return driver


def wait_clickable(driver, locator, timeout=5):
    return WebDriverWait(driver, timeout).until(EC.element_to_be_clickable(locator))


def wait_presence(driver, locator, timeout=8):
    return WebDriverWait(driver, timeout).until(EC.presence_of_element_located(locator))


def extract_siren_visible(html: str) -> set[str]:
    """Renvoie les SIREN visibles au format 'XXX XXX XXX' dans le HTML, sans ceux passés en href."""
    # regex pour 3 groupes de 3 chiffres séparés par espace ou insécable
    pattern = r"\b(\d{3}[ \u00A0]\d{3}[ \u00A0]\d{3})\b"
    found = re.findall(pattern, html)
    # enlever espaces pour ne conserver que les 9 chiffres
    return {siren.replace(" ", "") for siren in found}
    
# ----------------------------------------------------------------------
# 3. TRAITEMENT PRINCIPAL
# ----------------------------------------------------------------------
def main():
    driver = build_driver()
    wait   = WebDriverWait(driver, 12)
    base   = "https://annuaire-entreprises.data.gouv.fr/"
    results = []  # liste des résultats pour CSV

    for siret in SIRETS:
        siren_origin = siret[:9]
        associes     = set()

        # --- 3.1 recherche établissement ------------------------------
        driver.get(base)
        search = wait_clickable(driver, (By.CSS_SELECTOR, "#search-input-input"))
        search.clear()
        search.send_keys(siret + Keys.ENTER)
        wait.until(EC.url_contains("/etablissement/"))

        # --- 3.2 onglet Dirigeants ------------------------------------
        try:
            tab_dirigeants = wait_clickable(driver, (By.LINK_TEXT, "Dirigeants"))
        except TimeoutException:
            # pas d'onglet Dirigeants, passer au siret suivant
            continue
        tab_dirigeants.click()
        
        # attendre le chargement complet du contenu
        wait_presence(driver, (By.TAG_NAME, "body"))

        # --- 3.3 extraction des SIREN sur la page Dirigeants -------
        page_sirens = extract_siren_visible(driver.page_source)
        found_here  = {s for s in page_sirens if s != siren_origin}
        if found_here:
            associes.update(found_here)

        # --- 3.4 liste des boutons "voir ses entreprises" -----------
        try:
            wait_clickable(driver, (By.PARTIAL_LINK_TEXT, "voir ses entreprises"))
            boutons = driver.find_elements(By.PARTIAL_LINK_TEXT, "voir ses entreprises")
        except Exception:
            boutons = []

        # --- 3.5 boucle sur chaque bouton ----------------------------
        for btn in boutons:
            
            lien = btn.get_attribute("href")
            # ouvrir dans un nouvel onglet
            driver.execute_script("window.open(arguments[0], '_blank');", lien)
            driver.switch_to.window(driver.window_handles[-1])

            # attendre que la page charge
            wait_presence(driver, (By.TAG_NAME, "body"))

            # extraire tous les SIREN sur cette page
            siren_trouves = extract_siren_visible(driver.page_source)
            new_assocs = {s for s in siren_trouves if s != siren_origin}
            if new_assocs:
                associes.update(new_assocs)

            # fermer l’onglet et revenir
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        # --- 3.6 préparation du résultat ------------------------------
        sirens_str = ";".join(sorted(associes)) if associes else "NA"
        results.append({"siret": str(siret), "siren_associes": sirens_str})
        print(f"• {siret} – entreprises associées: {sorted(associes)}")

    
    # --- 3.7 écriture CSV final -------------------------------------
    with open(OUTFILE, "w", newline="", encoding="utf-8") as fp:
        writer = csv.DictWriter(fp, fieldnames=["siret", "siren_associes"])
        writer.writeheader()
        writer.writerows(results)

    driver.quit()

if __name__ == "__main__":
    main()


• 52811411900027 – entreprises associées: ['317784247', '492987821', '511574303', '538263955', '801536939', '803029156']
